## Inflating the biological assembly using symmetry operators
A structure downloaded from the protein databank may not contain all the coordinates for the biological assembly.  This is often an issue for molecules that act as multimers.  In this notebook, we will pull a symmetry operation from the PDB, by hand, to generate the symmetry related coordinates. 

This will be implemented into HackaMol, to make easier.

#### Author: Demian 

In [1]:
use Modern::Perl;
use HackaMol;
use Math::Vector::Real;
my $mol = HackaMol->new->pdbid_mol('3RVY'); 

HackaMol::Molecule=HASH(0x7fb052e023c8)


Warning: 3RVY.pdb exists, set self->overwrite(1) to overwrite at /Users/riccade/perl5/perlbrew/perls/p5.22.1t/lib/site_perl/5.22.1/HackaMol.pm line 26.


### slice out the matrices, and displacement vectors into a datastructure

In [2]:
my $symops = '
REMARK 350 APPLY THE FOLLOWING TO CHAINS: A, B                                  
REMARK 350   BIOMT1   1  1.000000  0.000000  0.000000        0.00000            
REMARK 350   BIOMT2   1  0.000000  1.000000  0.000000        0.00000            
REMARK 350   BIOMT3   1  0.000000  0.000000  1.000000        0.00000            
REMARK 350   BIOMT1   2 -1.000000  0.000000  0.000000     -125.59400            
REMARK 350   BIOMT2   2  0.000000 -1.000000  0.000000     -125.48300            
REMARK 350   BIOMT3   2  0.000000  0.000000  1.000000        0.00000    
'; # from the pdb

my %sym_op = (); # a hash to store them!
foreach my $line ( grep {m/BIOMT/} split( '\n' , $symops ) ){
    my @entries = split(' ', $line);
    push @{$sym_op{$entries[3]}}, V(@entries[4,5,6,7]);
}
#use Data::Dumper;
#print Dumper \%sym_op;

In [3]:
foreach my $symop (keys %sym_op){
    my @mat_d = @{$sym_op{$symop}};
    my $cx = V(map{$_->[0]} @mat_d); 
    my $cy = V(map{$_->[1]} @mat_d);
    my $cz = V(map{$_->[2]} @mat_d);
    my $dxyz = V(map{$_->[3]} @mat_d);
    foreach my $atom ($mol->all_atoms){
        my ($x,$y,$z) = @{$atom->xyz};
        my $xyz_new = $x*$cx + $y*$cy + $z*$cz + $dxyz;  
        $atom->push_coords($xyz_new);
    }    
}

In [4]:
$mol->t(1);
$mol->print_pdb('3RVY_1.pdb'); 
$mol->t(2);
$mol->print_pdb('3RVY_2.pdb'); 
1;

1


Warning: overwrite 3RVY_1.pdb at reply input line 2.

overwrite 3RVY_2.pdb at reply input line 4.


#### You may now load those two with 
$vmdpath/vmd -m 3RVY_1.pdb 3RVY_2.pdb